## Setup

# About

Analyse (a custom formated) http-request log for request duration.

Start with `jupyter notebook`.


## Example

![All requests](README.inc/all_requests.png)

![Just operation G](README.inc/operation_g_requests.png)

## Data format

The example dataset is split into the following columns

| Name | format | description |
|---|---|---|
| Timestamp  | timestamp  | Occurence of the request  |
| URL  | string | URL of the request (ignored!)  |
| command  | string  | The command executed. This is the grouping criteria for analysis.  |
| duration  | int  | Duration  of the request.  |

The unit of `duration` can be configured in `DURATION_UNIT`.

```csv
14/Jan/2019:03:46:03 /example/url operation_G 0
14/Jan/2019:03:46:07 /example/url operation_G 0
14/Jan/2019:03:46:07 /example/url operation_G 2
14/Jan/2019:03:46:08 /example/url operation_G 0
14/Jan/2019:03:46:09 /example/url operation_G 0
14/Jan/2019:03:46:10 /example/url operation_B 5
14/Jan/2019:03:46:19 /example/url operation_A 0
14/Jan/2019:03:46:19 /example/url operation_F 90
14/Jan/2019:03:46:20 /example/url operation_E 9
14/Jan/2019:03:46:24 /example/url operation_F 0
```

## Contributing

I am hosted at [GitHub](https://github.com/neuhalje/analyse_request_latency)!

## config

### Data format

The unit of the `duration` column is used as exmpanatory text in various charts. It is not interpreted in any way.

```python
# The unit of the duration column - only used in labeling charts
DURATION_UNIT = "s"
```


### Filtering

The source data set can be filtered prior sampling. This is especially useful only a subset of the calls are relevant. E.g. remove longpolling URLs because they would skew the statistics.


```python
# None: Do not filter at all
QUERY_FILTER=None

# Filter for all commands named 'myCommand'
QUERY_FILTER="command == 'myCommand'"

# Do some more elaborated filtering in Python

# Remove all urls that contain the string 'Lazy'
QUERY_FILTER=lambda df : np.logical_not(df.url.str.match('.*Lazy.*'))

# E.g. all commands that do not include 'G' and whose URL starts with ''/example'
QUERY_FILTER=lambda df : np.logical_not(df.command.str.match('.*G.*')) & df.url.str.match(r'^/example')
```

### Sampling

After filtering the data is sampled. Meaning: only a subset of the data is used for charting.  This considerably speeds up sampling. Especially the [plotly](https://plot.ly) charts benefit from a reduced dataset as they are rendered in the browser. Depending on your setup a chart with more than 100k elements will come close to freezing your browser.

With a large enough sampling the loss of precission os neglectable.

```python
# 'None' disables sampling.
MAX_ELEMENT_COUNT=None

# Take at most 100.000 elements.
MAX_ELEMENT_COUNT=100_000
```

### Massage the data

Outliers can make charts unusable, especially when they are magnitudes larger than the average data point. For getting an impression of the performance it is often good enough to know when a measurement is 'just too high'.

#### y-axis

This notebook provides two tunables for outlier filtering for the duration (y-axis):
* `PERCENTILE_LIMIT` (e.g. 0.999) sets all value higher than the (in this case 99.9%) percentile to the value of the percentile.
* `DURATION_LIMIT` is an additional upper limit.

Say `DURATION_LIMIT` is `100` and the calculated percentile of `PERCENTILE_LIMIT` is `101` then all values will be capped at `100`.

Say `DURATION_LIMIT` is `120` and the calculated percentile of `PERCENTILE_LIMIT` is `101` then all values will be capped at `101`. 

```python
# Cap y-axis to the 99.9th percentile or 60, whichever is lower.
PERCENTILE_LIMIT=0.999
DURATION_LIMIT=60  # unit: the same unit as used in the data files
```

#### x-axis

In order to control the resolution of the x-axis data can be grouped in buckets.

```python
# https://pandas.pydata.org/pandas-docs/stable/timeseries.html#timeseries-offset-aliases
ROUND_TO="15min"

ROUND_TO="6H"

ROUND_TO="1min"
```


In [ ]:
QUERY_FILTER=lambda df : np.logical_not(df.command.str.match('.*G.*')) & df.url.str.match(r'^/example')
PERCENTILE_LIMIT=0.999
DURATION_LIMIT=60 
ROUND_TO="15min"
DURATION_UNIT = "s"
MAX_ELEMENT_COUNT=100_000

_DATASET="example_dataset.txt"

#_DATASET="../combined-sorted_Jan_31-Feb_06.txt"
#_DATASET="../combined-sorted-Jan_14-Jan_22.txt"

DATASET=_DATASET

## Red Tape

In [ ]:
#!pip3 install scipy plotly pandas matplotlib seaborn

In [ ]:
%matplotlib inline

import matplotlib
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set(style="darkgrid")

from plotly.offline import download_plotlyjs, init_notebook_mode, plot,iplot
import plotly.plotly as py
import plotly.tools as tls

#Always run this the command before at the start of notebook (for Plotly)
init_notebook_mode(connected=True)


import plotly.graph_objs as go

def configure_figure_size():
    matplotlib.rcParams['figure.figsize'] = [15, 10]

In [ ]:
%%javascript
//  This is unsupported but increases the size of the output. Needed to really see the heatmaps
IPython.OutputArea.auto_scroll_threshold = 9999;

## Dataset

In [ ]:
df = pd.read_csv(DATASET, sep=" ")
df.columns = ['ts', 'url', 'command', 'duration']
df.head()

### Initial datase information

In [ ]:
df.command.value_counts()

In [ ]:
df.duration.describe(percentiles=[.25, .5, .75, .9, .95, .99, .999,.9999])

In [ ]:
df.duration.value_counts()

### Reduce dataset 

Filter the dataset, take a sampling (for faster processing)

In [ ]:
if QUERY_FILTER:
    if isinstance(QUERY_FILTER, str):
        df = df.query(QUERY_FILTER)
    else:
        df = df[QUERY_FILTER]

if MAX_ELEMENT_COUNT:
     df = df.sample(n=min(MAX_ELEMENT_COUNT, len(df.index)))

df.head()

### Convert data

You can customize the format of the timestamp here.

In [ ]:
df['timestamp'] = pd.to_datetime(df.ts,format="%d/%b/%Y:%H:%M:%S")

# put all requests in bins (e.g. 15min bins)
df['approx_ts'] = df['timestamp'].dt.round(ROUND_TO)  
df.sort_values(by='timestamp', inplace=True)
df.head()

### Clip outliers

Two bounds are put on the lateny:
* an absolute bound of `DURATION_LIMIT` seconds
* the `PERCENTILE_LIMIT`  (e.g. 0.999) which is calculated from the dataset

In [ ]:
df.duration.describe(percentiles=[.25, .5, .75, .9, .95, .99, .999,.9999])

In [ ]:
def clip_dataset(df):
    q = df.duration.quantile(q=PERCENTILE_LIMIT)
    latency_clipped_at = min(DURATION_LIMIT, q)
    df.duration.clip_upper(latency_clipped_at, inplace=True)
    return latency_clipped_at

latency_clipped_at = clip_dataset(df)

### Create aggregations

#### Add a counter column
This is used in aggregations.

In [ ]:
df['count'] = 1

## Analysis

#### Calculate the most costly calls

This calculation is based on the filtered (command), clipped (outliers) and sampled (number of items) dataset. In any case it should give a good indication which calls should be considered for optimisations.

Depending on the unit of the latency the averages can be misleading, esp. if the unit is [s] and all calls <500ms are logged as 0.

In [ ]:
def calculate_aggregated_call_cost(df):
    grouped = df.groupby(['command'], as_index=False)
    aggregated = grouped['duration'].agg([np.size, np.sum, np.average], as_index=False)
    aggregated.sort_values('sum', inplace=True, ascending=False)
    return aggregated

def plot_aggregated_call_cost(aggregated_call_cost):
    text_labels = [ f"""{command}: {count:_.0f} calls totalling in {sum:_.0f}{DURATION_UNIT} w. an average latency of {average:.1f}{DURATION_UNIT}"""  
                   for command, sum, count, average in  
                       zip(aggregated_call_cost.index,
                           aggregated_call_cost['sum'],
                           aggregated_call_cost['size'],
                           aggregated_call_cost['average'])]    
    
    data = [go.Bar(
            x=aggregated_call_cost.index,
            y=aggregated_call_cost['sum'],
            text  = text_labels
    )]

    layout= go.Layout(
            title= "Absolute call cost per command",
            hovermode= 'closest',
            xaxis= dict(
                title= 'Operation',
                ticklen= 5,
                zeroline= False,
        ),
        yaxis=dict(
            title= f'Aggregated call time in [{DURATION_UNIT}]',
            ticklen= 5,
            gridwidth= 2,
        ),
        showlegend= False
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)
    
#calculate_aggregated_call_cost(df).head()  
plot_aggregated_call_cost(calculate_aggregated_call_cost(df))


In [ ]:
df.timestamp.describe()

### Show the distribution of  latency

In [ ]:
def distribution_of_duration(df):
    configure_figure_size()
    sns.distplot(df.duration, kde=False)
    
distribution_of_duration(df)

### Latency Heatmap

Show latency as a heatmap with time on the x-axis, latency on the y-axis and color frequency of measurements.

In [ ]:
def aggregate_for_heatmap(df):
    grouped = df.groupby(['approx_ts','duration'], as_index=False)
    aggregated = grouped['count'].agg(np.size)
    return aggregated
    
def plot_latency_heatmap(aggregated, command):
    call_count = aggregated['count'].sum()
    
    if command:
        title = f'Latency of {call_count:_} "{command}" calls in [{DURATION_UNIT}]'
    else:
        # not filtered
        title = f'Latency of {call_count:_} calls in [{DURATION_UNIT}]'

    #
    #  Call count
    #
    
    call_count_by_date = aggregated[['approx_ts','count']].groupby(['approx_ts'], as_index = False)['count'].agg('sum')
    
    text_labels_count  = [ f"""{when} - {count:_} calls total"""  
                   for when, count in  
                       zip(call_count_by_date['approx_ts'],
                           call_count_by_date['count'])]
                          
    call_count_trace = go.Scatter(
        name="call count",
        x = call_count_by_date['approx_ts'],
        y = call_count_by_date['count'],
        text  = text_labels_count,
        mode = 'lines+markers',
        yaxis='y',
    )

    #
    # latency heatmap 
    #
    
    text_labels_heatmap = [ f"""{when} - {count:_} call(s) w. {latency:_.1f}{DURATION_UNIT} latency"""  
                   for when, latency,count in  
                       zip(aggregated['approx_ts'],
                           aggregated['duration'],
                           aggregated['count'])
                  ]

    heatmap_trace = go.Scatter(
        name="latency distribution",
        x = aggregated['approx_ts'],
        y = aggregated['duration'],
        text  = text_labels_heatmap,
        mode ='markers',
        marker=dict(
            color = aggregated['count'],
            colorscale = 'Jet',
            showscale = False,
            symbol = "square"
        ),
        yaxis='y2'
    )
      


    #
    #  Layout
    #
    
    layout = go.Layout(
        title = title,
        hovermode = 'closest',
        xaxis = dict(
                title = 'Timestamp',
                ticklen = 5,
                zeroline = True,
                gridwidth = 2,
        ),

        yaxis = dict(
            title = f'Call count',
            side='left',
        ),
         yaxis2 = dict(
            title = f'Latency in [{DURATION_UNIT}] (capped at {latency_clipped_at:_.1f}{DURATION_UNIT})',
            side='right',
            overlaying='y',
    ),
        showlegend= True
    )

    data = [call_count_trace,heatmap_trace]
    fig = go.Figure(data = data, layout = layout)

    iplot(fig)

plot_latency_heatmap(aggregate_for_heatmap(df), command = None)    
for command in df['command'].unique():
    filtered_df = df.query("command == @command")
    filtered_aggregate = aggregate_for_heatmap(filtered_df)
    plot_latency_heatmap(filtered_aggregate, command)

